In [ ]:

import sys
import ee
ee.Authenticate()
ee.Initialize()

# check if GEE is already imported to avoid requesting authenticatiation multiple times
modulename = 'Masks'
if modulename not in sys.modules:
    # import Masks Class
    %run Masks.ipynb
# else:
    # Mask class already imported and authenticated


# check if GEE is already imported to avoid requesting authenticatiation multiple times
modulename = 'ee'
if modulename not in sys.modules:
    # import GEE and Authenticate, token or log in will be asked from web browser
    import ee
    # ee.Authenticate()
    ee.Initialize()
# else:
    # google earth engine already imported and authenticated



# check if GEE is already imported to avoid requesting authenticatiation multiple times
modulename = 'ipynb_Sentinel1'
if modulename not in sys.modules:
    %run Sentinel1.ipynb
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['ipynb_Sentinel1'] = None 
#else
   # Sentinel1 modules has already been loaded somewhere else


# Visualisation
modulename = 'ipynb_MapVisualisation'
if modulename not in sys.modules:
    %run MapVisualisation.ipynb
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['ipynb_MapVisualisation'] = None 
#else
   # MapVisualisation module has already been loaded somewhere else


In [ ]:
startDate = '2017-01-01'
endDate   = '2017-12-31'

countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
geometry = countries.filter(ee.Filter.eq('country_na', 'Cyprus'))

fstartDate = '2001-01-01'
fendDate   = '2022-12-31'

masks = {
    'gsw': 300, 
    'lmask': 30, 
    #'aspectAsc':0,    
    'forestMask': {
        'buffer':30, 
        'startDate':fstartDate, 
        'endDate':fendDate}
        } #,'aspectDes':0, # 'aspectAsc':0}
#masks = {}

s1 = Sentinel1(geometry,startDate,endDate,masks,False)

# remove images from the collection 
# in Sentinel 1 there is an internal removePeriod command for removing the selected period 
# for all VV, VH asc and desc respectively 
# This is why the removePeriodFromCollection() function is not used
s1.removePeriod(startDate,'2017-01-10')
s1.byMonth(2017)

VVAsc = s1.getVVAsc()
VHAsc = s1.getVHAsc()
VVDes = s1.getVVDes()
VHDes = s1.getVHDes()



VVAscMedian = s1.getVVAscMedian()
VHAscMedian = s1.getVHAscMedian()
VVDesMedian = s1.getVVDesMedian()
VHDesMedian = s1.getVHDesMedian()



In [ ]:

# Set visualization parameters.
#vis_params = {
#  'min': 0,
#  'max': 4000,
#  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[33,34], zoom_start=6)


# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)


s1.byMonth(2017)

# Add VVAsc to the map
my_map.add_ee_layer(VVAscMedian, {'min':-30,'max':0}, 'VVAscMedian')
# Add VHAsc tp the map 
my_map.add_ee_layer(VHAscMedian, {'min':-30,'max':0}, 'VHAscMedian')
# Add VVDes to the map 
my_map.add_ee_layer(VVDesMedian, {'min':-30,'max':0}, 'VVDesMedian')
# Add descending aspects map to the map
my_map.add_ee_layer(VHDesMedian, {'min':-30,'max':0}, 'VHDesMedian')


# remove images from the collection 
# NEVER apply this to a single aspect, always call internal Sentinel 1 member function
# to be applied to all VVAsc, VHAsc, VVDes, VHDes
# VHAsc = removePeriodFromCollection(VHAsc,startDate,'2017-04-30')
# ** Use this method for removing period instead before requesting the data*** 
# s1 = s1.removePeriod(startDate,'2017-04-30')

#VV12 = VVDes.filter(ee.Filter.calendarRange(1, 3, 'month'))#.mean().set('month',1)
#my_map.add_ee_layer(VV12, {'min':-30,'max':0}, 'VV12')

#print(VHAsc.getInfo())

#print("No remaining images: ", getNoOfBands(VVAsc))
# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())


plugins.Fullscreen().add_to(my_map)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

